In [1]:
import textgrad as tg
from textgrad.engine import get_engine
from textgrad.variable import Variable
from textgrad.optimizer import TextualGradientDescent
from textgrad.verifier import TextualVerifierV4
from textgrad.loss import TextLoss

### Setup LLM Engine

In [2]:
engine = get_engine("gemini-1.5-pro")
tg.set_backward_engine("gemini-1.5-pro")

/Users/eugeniusms/Development/SKRIPSI/sevet/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setup Solution to Optimize

In [3]:
initial_solution = """To solve the equation 3x^2 - 7x + 2 = 0, we use the quadratic formula:
x = (-b ± √(b^2 - 4ac)) / 2a
a = 3, b = -7, c = 2
x = (7 ± √((-7)^2 + 4(3)(2))) / 6
x = (7 ± √73) / 6
The solutions are:
x1 = (7 + √73)
x2 = (7 - √73)"""

solution = Variable(initial_solution,
                       requires_grad=True,
                       role_description="solution to the math question")


### Set Loss Variable

In [4]:
loss_system_prompt = Variable("""You will evaluate a solution to a math question. 
Do not attempt to solve it yourself, do not give a solution, only identify errors. Be super concise.""",
                              requires_grad=False,
                              role_description="system prompt")


### Inisiate Optimizer

In [5]:
optimizer = TextualGradientDescent([solution])

### Find Loss

In [6]:
loss = TextLoss(loss_system_prompt, engine=engine)
result = loss(solution) # Forward method in Loss Function

print("INITIAL LOSS:", result)

INITIAL LOSS: The calculation of  b² - 4ac was incorrect: it should be (-7)² - 4 * 3 * 2 = 49 - 24 = 25, not 49 + 24 = 73.  Also, the division by 6 is missing in the final answers for x1 and x2.



### Using TextualVerifier to Verify Loss

In [7]:
# Verify Loss
verifier = TextualVerifierV4(verifier_engine=engine, step_eval_iterations=3, logger=False)
verified_result = verifier.verify(instance=solution, 
                                    prompt=loss_system_prompt,
                                    calculation=result)

print("VERIFIED RESULT:", verified_result.value)

VERIFIED RESULT: The solution lacks explicit steps for verifying the correct identification and substitution of coefficients (a, b, and c).  It also doesn't emphasize the importance of handling signed values, particularly when calculating b² and 4ac, increasing the risk of sign errors.  A more systematic approach involving clearly identifying, substituting, and verifying each coefficient, along with explicit calculations including signs, would improve accuracy and prevent common errors.


### Optimize Solution with Optimizer (TGD)

In [8]:
# TRANSFER: Replace the loss value with verified result
result.set_value(verified_result.value) 

print("FINAL LOSS (after transfer):", result.value)

FINAL LOSS (after transfer): The solution lacks explicit steps for verifying the correct identification and substitution of coefficients (a, b, and c).  It also doesn't emphasize the importance of handling signed values, particularly when calculating b² and 4ac, increasing the risk of sign errors.  A more systematic approach involving clearly identifying, substituting, and verifying each coefficient, along with explicit calculations including signs, would improve accuracy and prevent common errors.


In [9]:
# Optimize
result.backward()

optimizer.step()
print("SOLUTION:", solution.value)

['Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \nDo not attempt to solve it yourself, do not give a solution, only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> To solve the equation 3x^2 - 7x + 2 = 0, we use the quadratic formula:\nx = (-b ± √(b^2 - 4ac)) / 2a\na = 3, b = -7, c = 2\nx = (7 ± √((-7)^2 + 4(3)(2))) / 6\nx = (7 ± √73) / 6\nThe solutions are:\nx1 = (7 + √73)\nx2 = (7 - √73) </LM_INPUT>\n\n<LM_OUTPUT> The solution lacks explicit steps for verifying the correct identification and substitution of coefficients (a, b, and c).  It also doesn\'t emphasize the importance of handling signed values, particularly when calculating b² and 4ac, increasing the risk of sign errors.  A more systematic approach involving clearly identifying, substituting, and verifying each coefficient, along with explicit calculations including signs, would improve accuracy and prevent common errors. </LM_OUTPUT>\n\n</CO